## Extending the capabilities of our model

An LLM is a very capable tool, but only to the extent of the knowledge or information it has been trained on. After all, you only know what you know, right? But what if you need to ask a question that is not in the training data? Or what if you need to ask a question that is not in the training data, but is related to it?

There are different ways to solve this problem, depending on the resources you have and the time or money you can spend on it. Here are a few options:

- Fully retrain the model to include the information you need. For an LLM, it's only possible for a handful of companies in the world that can afford literally thousands of GPUs running for weeks.
- Fine-tune the model with this new information. This requires way less resources, and can usually be done in a few hours or minutes (depending on the size of the model). However as it does not fully retrain the model, the new information may not be completely integrated in the answers. Fine-tuning excels at giving a better understanding of a specific context or vocabulary, a little bit less on injecting new knowledge. Plus you have to retrain and redeploy the model anyway any time you want to add more information.
- Put this new information in a database and have the parts relevant to the query retrieved and added to this query as a context before sending it to the LLM. This technique is called **Retrieval Augmented Generation, or RAG**. It is interesting as you don't have to retrain or fine-tune the model to benefit of this new knowledge, that you can easily update at any time.

We have already prepared a Vector Database using [Milvus](https://milvus.io/), where we have stored (in the form of [Embeddings](https://www.ibm.com/topics/embedding)) the content of the [California Driver's Handbook](https://www.dmv.ca.gov/portal/handbook/california-driver-handbook/).

In this Notebook, we are going to use RAG to **make some queries about a Claim** and see how this new knowledge can help without having to modify our LLM.

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages.

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.chains import LLMChain, RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.llms import VLLMOpenAI
from langchain_community.vectorstores import Milvus
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from milvus_retriever_with_score_threshold import MilvusRetrieverWithScoreThreshold

#Turn off warnings when downloading the embedding model
import transformers
transformers.logging.set_verbosity_error()

### Langchain elements

Again, we are going to use Langchain to define our task pipeline.

First, the **LLM** where we will send our queries.

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:8000"

# LLM definition
llm = VLLMOpenAI(           # we are using the vLLM OpenAI-compatible API client. But the Model is running on OpenShift, not OpenAI.
    openai_api_key="EMPTY",   # and that is why we don't need an OpenAI key for this.
    openai_api_base= f"{inference_server_url}/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

Then the connection to the **vector database** where we have prepared and stored the California Driver Handbook.

In [ ]:
# First we define the embeddings that we used to process the Handbook
model_kwargs = {"trust_remote_code": True}
embeddings = HuggingFaceEmbeddings(
            model_name="nomic-ai/nomic-embed-text-v1",
            model_kwargs=model_kwargs,
            show_progress=False,
        )

# Then we define the retriever that will fetch the relevant data from the Milvus vector store
retriever = MilvusRetrieverWithScoreThreshold(
            embedding_function=embeddings,
            collection_name="california_driver_handbook_1_0",
            collection_description="",
            collection_properties=None,
            connection_args={
                "host": "vectordb-milvus.ic-shared-milvus.svc.cluster.local",
                "port": "19530",
                "user": "root",
                "password": "Milvus",
            },
            consistency_level="Session",
            search_params=None,
            k=4,
            score_threshold=0.99,
            metadata_field="metadata",
            text_field="page_content",
        )

We will now define the **template** to use to make our query. Note that this template now contains a **References** section. That's were the documents returned from the vector database will be injected.

In [ ]:
template="""<s>[INST]
You are a helpful, respectful and honest assistant named "Parasol Assistant".
You will be given a claim summary, references to provide you with information, and a question.
You must answer the question based as much as possible on this claim with the help of the references.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>

Claim Summary:
{claim}

References:
{{context}}

Question: {{question}} [/INST]
"""

We are now ready to query the model!

In the `claims` folder we have JSON files with examples of claims that could be received. We are going to read the first claim and ask a question related to it.

In [ ]:
# Read the claim and put its content in the "claim" variable

filename = 'claims/claim1.json'

# Opening JSON file
with open(filename, 'r') as file:
    data = json.load(file)
claim = data["content"]

### First test, no additional knowledge

Let's start with a first query about the claim, but without help from our vector database.

In [ ]:
# Create and send our query.

query = "Was Daniel allowed to pass at the red light?"

# Quick hack to reuse the same template with a different type of query.
prompt_template = template.format(claim=claim).format(context="", question=query)
prompt = PromptTemplate.from_template(prompt_template)
conversation = LLMChain(
            llm=llm,
            prompt=prompt,
            verbose=False
        )
resp = conversation.predict(input="")

We can see that the answer is valid. Here the model is using its general understanding of traffic regulation.

### Second test, with added knowledge

We will use the same prompt and query, but this time the model will have access to some references from the California's Driver Handbook.

In [ ]:
# Create and send our query.

query = "Was Daniel allowed to pass at the red light?"

prompt_template = template.format(claim=claim)
prompt = PromptTemplate.from_template(prompt_template)
rag_chain = RetrievalQA.from_chain_type(
            llm,
            retriever=retriever,
            chain_type_kwargs={"prompt": prompt},
            return_source_documents=True,
        )
resp = rag_chain.invoke({"query": query})

That is pretty neat! Now the model refers directly to a source stating that **a red traffic signal light means "STOP."**.

But where did we get this information from? We can look into the sources associated with the answers from the vector database.

In [ ]:
def format_sources(input_list):
        sources = ""
        if len(input_list) != 0:
            sources += input_list[0].metadata["source"] + ', page: ' + str(input_list[0].metadata["page"])
            page_list = [input_list[0].metadata["page"]]
            for item in input_list:
                if item.metadata["page"] not in page_list: # Avoid duplicates
                    page_list.append(item.metadata["page"])
                    sources += ', ' + str(item.metadata["page"])
        return sources

results = format_sources(resp['source_documents'])

print(results)

That's it! We now know how to complement our LLM with some external knowledge!